In [1]:
from __future__ import print_function
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import os
import sys
from IPython.display import display, Image
from six.moves import cPickle as pickle
import tensorflow as tf


In [2]:
#load data first
pickle_file = '/home/htan/proj/TensorFlow/data/visibility/' +  'ASOS_alone.pickle'

with open(pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset= save['train_dataset']
    validate_dataset = save['validate_dataset']
    test_dataset = save['test_dataset']
    test_old = save['test_dataset_evan']
    train_old = save['train_dataset_evan']
    del save

train_time = train_dataset['time']
train_data = train_dataset['data']
train_label = train_dataset['label']
validate_time = validate_dataset['time']
validate_data = validate_dataset['data']
validate_label = validate_dataset['label']
test_time = test_dataset['time']
test_data = test_dataset['data']
test_label = test_dataset['label']
test_old_data = test_old['data']
test_old_label = test_old['label']
train_old_data = train_old['data']
train_old_label = train_old['label']


In [3]:
#dataset normalize
mean = train_data.mean(axis = 0)
std = train_data.std(axis = 0)
print(mean.shape, std.shape)
train_data_n = (train_data - mean)/std
validate_data_n = (validate_data - mean)/std
test_data_n = (test_data - mean)/std
test_size = test_label.shape[0]
batch_size = 128

(70,) (70,)


In [5]:
# write create simple linear regression model
graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.placeholder(tf.float32, shape=(128, 70))
    tf_train_label = tf.placeholder(tf.float32, shape=(128))
    tf_validate_dataset = tf.constant(validate_data_n.astype(np.float32)[:, :])
    tf_validate_label = tf.constant(validate_label.astype(np.float32)[:])
    tf_test_dataset1 = tf.constant(test_data_n.astype(np.float32)[:30000, :])
    tf_test_label1 = tf.constant(test_label.astype(np.float32)[:30000])
    tf_test_dataset2 = tf.constant(test_data_n.astype(np.float32)[30000:, :])
    tf_test_label2 = tf.constant(test_label.astype(np.float32)[30000:])
    weights = tf.Variable(tf.truncated_normal([train_data_n.shape[1], 1]))
    biases = tf.Variable(tf.zeros([1]))

    def model(X, w, b):
        return tf.matmul(X, w) + b

    predicted_label = model(tf_train_dataset, weights, biases)
    loss = tf.reduce_mean(tf.square(predicted_label - tf_train_label))

    # Learning rate decay
    global_step = tf.Variable(0)
    starter_learning_rate = 0.01
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step, 500, 0.90, staircase=True)
    op = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step = global_step)


    train_loss = tf.reduce_mean(tf.abs(predicted_label - tf_train_label))
    test_loss1 = tf.reduce_mean(tf.abs(model(tf_test_dataset1, weights, biases) - tf_test_label1))
    test_loss2 = tf.reduce_mean(tf.abs(model(tf_test_dataset2, weights, biases) - tf_test_label2))
    test_loss = (test_loss1 * 30000 + test_loss2 * (test_size - 30000)) / test_size
    validate_loss = tf.reduce_mean(tf.abs(model(tf_validate_dataset, weights, biases) - tf_validate_label))



In [6]:
#train the model with stochastic gradient descent training
num_steps = 25000
with tf.Session(graph = graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    for step in range(num_steps):
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_label.shape[0] - batch_size)
        # Generate a minibatch.
        batch_data = train_data_n[offset:(offset + batch_size), :]
        #print(batch_data.shape)
        batch_labels = train_label[offset:(offset + batch_size)]
        feed_dict = {tf_train_dataset : batch_data, tf_train_label : batch_labels}
        #session.run(predicted_label, feed_dict=feed_dict)
        _, l, _, r = session.run([predicted_label, loss, op, learning_rate], feed_dict=feed_dict)

        if (step % 10 == 0):
            print('Loss at step %d: LR %f MSE %f MAE %f VALIDATE MAE %f' % (step, r, l, train_loss.eval(feed_dict=feed_dict), validate_loss.eval()))
    print('test1, test2, test MAE: %.3f, %.3f, %.3f' % (test_loss1.eval(), test_loss2.eval(), test_loss.eval()))
 

Initialized
Loss at step 0: LR 0.010000 MSE 222.307800 MAE 10.022051 VALIDATE MAE 3.481551
Loss at step 10: LR 0.010000 MSE 43.774975 MAE 4.725509 VALIDATE MAE 3.359485
Loss at step 20: LR 0.010000 MSE 56.509937 MAE 4.208092 VALIDATE MAE 3.359485
Loss at step 30: LR 0.010000 MSE 8.237055 MAE 2.225244 VALIDATE MAE 3.359485
Loss at step 40: LR 0.010000 MSE 6.511482 MAE 1.712731 VALIDATE MAE 3.359485
Loss at step 50: LR 0.010000 MSE 6.175277 MAE 1.949716 VALIDATE MAE 3.359485
Loss at step 60: LR 0.010000 MSE 4.829509 MAE 1.287946 VALIDATE MAE 3.081450
Loss at step 70: LR 0.010000 MSE 33.778259 MAE 4.592544 VALIDATE MAE 3.359485
Loss at step 80: LR 0.010000 MSE 23.712811 MAE 3.618443 VALIDATE MAE 3.359485
Loss at step 90: LR 0.010000 MSE 28.047668 MAE 3.262958 VALIDATE MAE 3.231786
Loss at step 100: LR 0.010000 MSE 12.806293 MAE 2.371156 VALIDATE MAE 1.729923
Loss at step 110: LR 0.010000 MSE 5.229412 MAE 1.702709 VALIDATE MAE 1.679743
Loss at step 120: LR 0.010000 MSE 13.494892 MAE 2.7125